In [130]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from unidecode import unidecode
from sklearn.decomposition import PCA
import numpy as np

In [80]:
def main():

    df = pd.read_csv("playerdata.csv")

    df.columns

    names = df['Player']

    df.drop(["Rk", "Pos", "Player", "Player-additional", "Unnamed: 19", "Unnamed: 24", "Tm"], axis=1, inplace=True)

    df.columns

    scaler = StandardScaler()

    df = pd.DataFrame(scaler.fit_transform(df))

    for i in df.columns[df.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
        df[i].fillna(df[i].mean(),inplace=True)

    pca = PCA(n_components=0.8)

    df = pd.DataFrame(pca.fit_transform(df))

    df["Name"] = names

    df

    def n_format(name):
        name = name.split(" ")
        name[0] = name[0][0]+'.'
        name = " ".join(name)
        return unidecode(name)

    df['Name'] = df['Name'].apply(lambda x: n_format(x))

    df.to_csv("playerdatapca.csv")

In [81]:
lineup = pd.read_csv("cleanedlineup.csv").drop(['Unnamed: 0'], axis=1)

In [101]:
main()
pdata = pd.read_csv("playerdatapca.csv", index_col="Name").drop(['Unnamed: 0'], axis=1)
pdata = pdata[~pdata.index.duplicated(keep='first')]
pdata.columns = ["pc1", "pc2", "pc3", "pc4", "pc5", "pc6", "pc7"]
comb = lineup.values.tolist()
for i in comb:
    for j in i[0].split(" - "):
        i.append(list(pdata.loc[j].squeeze()))

In [105]:
comb = pd.DataFrame(comb)

In [109]:
comb.columns = ["Name", "Y_VAL", "P1", "P2", "P3", "P4", "P5"]

In [111]:
comb['X_VAL'] = comb['P1'] + comb['P2'] + comb['P3'] + comb['P4'] + comb['P5']

In [118]:
comb = comb[['Name', 'Y_VAL', 'X_VAL']]

In [120]:
len(comb['X_VAL'][0])

35

In [122]:
df2 = pd.DataFrame(comb.X_VAL.tolist(), index=comb.index)

In [124]:
comb = pd.concat([comb, df2], axis=1, join="inner")

In [127]:
names = comb['Name']

In [131]:
comb.to_csv("finalized_data_model.csv", index=False)

In [128]:
X, y = comb.drop(['Name', 'Y_VAL', 'X_VAL'], axis=1), comb[['Y_VAL']]

In [129]:
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

NameError: name 'np' is not defined